# Максимизация R2

Метрика R2 понятнее RMSE. Остановимся на коэффициенте детерминации и подберём лучшую модель. Чтобы решить задачу успешно, доведем метрику до `0.14`.

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from joblib import dump

In [3]:
data = pd.read_csv('flights_preprocessed.csv')

RND_STATE = 12345

target = data['Arrival Delay']
features = data.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RND_STATE)

Вычислим значение R2 функцией `score()` у модели. Это по умолчанию метрика для моделей регрессии в sklearn.

In [4]:
model = LinearRegression()
model.fit(features_train, target_train)
model.score(features_valid, target_valid)

0.09710497146204988

Подберем подходящую глубину дерева сначала при небольшом числе деревьев. С их увеличением качество модели всегда растёт. Но увеличивается и время обучения.

In [5]:
for depth in range(1, 16, 1):
    model = RandomForestRegressor(n_estimators=20, max_depth=depth, random_state=RND_STATE)
    model.fit(features_train, target_train)
    print(f"depth {depth}: {model.score(features_valid, target_valid)}")

depth 1: 0.037285907526148354
depth 2: 0.06819642209621124
depth 3: 0.08784043439422196
depth 4: 0.10410539417320942
depth 5: 0.1170148926305088
depth 6: 0.12812270304109197
depth 7: 0.14023988572891255
depth 8: 0.1509905046331984
depth 9: 0.1561784744742989
depth 10: 0.15937461353477067
depth 11: 0.16165140416942136
depth 12: 0.16050981198241898
depth 13: 0.15524848318843387
depth 14: 0.14612375630805674
depth 15: 0.1419794076464067


Теперь запустим обучение случайного леса с большим количеством деревьев:

In [6]:
%%time

model = RandomForestRegressor(n_estimators=100, max_depth=11, random_state=RND_STATE)
model.fit(features_train, target_train)
print("train:", model.score(features_train, target_train))
print("valid:", model.score(features_valid, target_valid))

train: 0.3784551354930391
valid: 0.16447073084035702
CPU times: user 9.29 s, sys: 13.1 ms, total: 9.3 s
Wall time: 9.31 s


In [7]:
dump(model, 'model.joblib')

['model.joblib']